In [ ]:
%pip install geopandas numpy pandas seaborn statsmodels

In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import statsmodels.api as sm

In [ ]:
gdf_tracts = gpd.read_file("https://github.com/dylanqcdi/PPD534FA23/raw/main/tl_2022_06_tract/tl_2022_06_tract.shp")
gdf_tracts.shape

In [ ]:
gdf_boundary = gpd.read_file("https://github.com/dylanqcdi/PPD534FA23/raw/main/City_Boundary/City_Boundary.shp")
gdf_boundary.shape

In [ ]:
gdf_tracts.crs == gdf_boundary.crs

In [ ]:
gdf_boundary = gdf_boundary.to_crs(gdf_tracts.crs)

In [ ]:
gdf_lacity = gpd.clip(gdf_tracts, gdf_boundary)
gdf_lacity.shape

In [ ]:
gdf_income1 = gpd.read_file("https://raw.githubusercontent.com/gboeing/ppd534/main/data/census_tracts_data_la.csv")
gdf_income = gdf_income1.rename(columns={"GEOID10": "GEOID"})
gdf_income.shape

In [ ]:
lacensus = pd.merge(gdf_lacity, gdf_income, how="left", left_on="GEOID", right_on="GEOID")
gdf_lacensus = gpd.GeoDataFrame(lacensus, geometry='geometry_x')
gdf_lacensus.set_geometry('geometry_x', inplace=True)
gdf_lacensus.crs = "EPSG:4326"
gdf_lacensus.shape

In [ ]:
df_collision = pd.read_csv("https://raw.githubusercontent.com/dylanqcdi/PPD534FA23/a8c54e8cb797d9266490e6473112b898b95c8d7e/Collision_Points_SCAG.csv")
df_collision["geometry"] = gpd.points_from_xy(x=df_collision["X"], y=df_collision["Y"])
gdf_collision = gpd.GeoDataFrame(df_collision, crs="epsg:4326")
gdf_collision.shape

In [ ]:
df_collision = pd.read_csv("https://raw.githubusercontent.com/dylanqcdi/PPD534FA23/a8c54e8cb797d9266490e6473112b898b95c8d7e/Collision_Points_SCAG.csv")
df_collision["geometry"] = gpd.points_from_xy(x=df_collision["X"], y=df_collision["Y"])
gdf_collision = gpd.GeoDataFrame(df_collision, crs="epsg:4326")
county = ["LOS ANGELES"]
gdf_collision = gdf_collision[gdf_collision["COUNTY"].isin(county)]
gdf_lacollision = gpd.clip(gdf_collision, gdf_lacensus)
gdf_lacollision.shape

In [ ]:
gdf = gpd.sjoin(gdf_lacensus, gdf_lacollision, how="left", predicate='contains')
gdf.shape

In [ ]:
gdf['med_household_income'] = gdf['med_household_income'].replace('\$', '', regex=True)
gdf['med_household_income'] = gdf['med_household_income'].replace('', np.nan)
gdf['med_household_income'] = gdf['med_household_income'].astype(float)

In [ ]:
attribute = ['COLLISION_SEVERITY', 'NUMBER_KILLED',
       'NUMBER_INJURED', 'COUNT_SEVERE_INJ', 'COUNT_VISIBLE_INJ',
       'COUNT_COMPLAINT_PAIN', 'COUNT_PED_KILLED', 'COUNT_PED_INJURED',
       'COUNT_BICYCLIST_KILLED', 'COUNT_BICYCLIST_INJURED', 'COUNT_MC_KILLED',
       'COUNT_MC_INJURED']
sum_collision = gdf.groupby("GEOID")[attribute].sum()
sum_collision.shape

In [ ]:
gdf = gdf.merge(sum_collision, left_on="GEOID",right_index=True, how="left")
gdf.shape

In [ ]:
gdf.columns

**Deadly Collision by Census Tract**

In [ ]:
ax = gdf.plot(column="NUMBER_KILLED_y",
                        cmap="plasma",
                        lw=0.2,
                        figsize=(9, 6),
                        legend=True)